In [1]:
import pandas as pd
import numpy
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

pd.set_option('chained_assignment',None)

In [2]:
file = pd.read_csv('data.csv')

In [3]:
sub_df = file[['Top1', 'Top1Summ1', 'Top1Summ2', 'Top1Keystone', 'Top1Xp', 'Top1Gold',
                     'Top2', 'Top2Summ1', 'Top2Summ2', 'Top2Keystone', 'Top2Xp', 'Top2Gold']]

sub_df['TopGoldDiff'] = (sub_df['Top1Gold'] * 10) - (sub_df['Top2Gold'] * 10)

x = sub_df[['Top1', 'Top1Summ1', 'Top1Summ2', 'Top1Keystone', 'Top2', 'Top2Summ1', 'Top2Summ2', 'Top2Keystone']]

g_diff = sub_df['TopGoldDiff']

In [10]:
x_train, x_test, y_train, y_test = train_test_split(x, g_diff, test_size=0.3, random_state=0)

testing_forest = RandomForestRegressor(random_state=0)
testing_forest.fit(x_train, y_train)

testing_tree = DecisionTreeRegressor(random_state=0)
testing_tree.fit(x_train, y_train)

y_forest = testing_forest.predict(x_test)
y_tree = testing_tree.predict(x_test)
forest_mse = mean_squared_error(y_test, y_forest)
print('Random Forest MSE : {}'.format(numpy.sqrt(forest_mse)))

tree_mse = mean_squared_error(y_test, y_tree)
print('Decision Tree MSE : {}'.format(numpy.sqrt(tree_mse)))

Random Forest MSE : 996.4406743288878
Decision Tree MSE : 1335.758573013837


In [20]:
forest = RandomForestRegressor(random_state=0)

param_grid = {
    'n_estimators' : [510],
    'max_depth' : [9],
    'max_features': ['sqrt'],
    'max_leaf_nodes': [115, 110],
    'criterion' : ['mse']
}

cv_grid = GridSearchCV(estimator=forest, param_grid=param_grid, cv=5, verbose=1, n_jobs=6)
cv_grid.fit(x_train, y_train)
cv_grid.best_params_

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  10 out of  10 | elapsed:    1.9s remaining:    0.0s
[Parallel(n_jobs=6)]: Done  10 out of  10 | elapsed:    1.9s finished


{'criterion': 'mse',
 'max_depth': 9,
 'max_features': 'sqrt',
 'max_leaf_nodes': 115,
 'n_estimators': 510}

In [21]:
model2 = RandomForestRegressor(n_estimators=cv_grid.best_params_['n_estimators'], max_depth=cv_grid.best_params_['max_depth'], max_leaf_nodes=cv_grid.best_params_['max_leaf_nodes'],
                               max_features=cv_grid.best_params_['max_features'], random_state=0)
model2.fit(x_train, y_train)

y2 = model2.predict(x_test)

mse2 = mean_squared_error(y_test, y2)
rmse = numpy.sqrt(mse2)
print(rmse)

scores = -1 * cross_val_score(model2, x, g_diff,
                        cv=5 , scoring="neg_mean_squared_error")
scores
numpy.sqrt(sum (scores) / len(scores))

946.1407125448807


944.8330693760631